In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

### Basic Dataframe Examination

<font size="3">Start off with a basic examination of the dataframes using `Pandas` standard EDA functions.</font>

In [2]:
species = pd.read_csv('data/species_info.csv')
observations = pd.read_csv('data/observations.csv')

In [3]:
species.head(3)

,category,scientific_name,common_names,conservation_status
0,Mammal,Clethrionomys gapperi gapperi,Gapper's Red-Backed Vole,NaN
1,Mammal,Bos bison,"American Bison, Bison",NaN
2,Mammal,Bos taurus,"Aurochs, Aurochs, Domestic Cattle (Feral), Dom...",NaN


In [4]:
observations.head(3)

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138


In [5]:
display(species.shape)
observations.shape

(5824, 4)

(23296, 3)

In [6]:
species.info()
print()
observations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5824 entries, 0 to 5823
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             5824 non-null   object
 1   scientific_name      5824 non-null   object
 2   common_names         5824 non-null   object
 3   conservation_status  191 non-null    object
dtypes: object(4)
memory usage: 182.1+ KB

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23296 entries, 0 to 23295
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   scientific_name  23296 non-null  object
 1   park_name        23296 non-null  object
 2   observations     23296 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 546.1+ KB


<font size="3">There are only null values in the species dataframe are under the column conservation status. Some more familiarization of the species data set would be helpful before making decision on the null values.<font size="3">

In [7]:
# species.conservation_status.value_counts()
con_status_counts = species.conservation_status.value_counts(dropna=False)
con_status_counts

NaN                   5633
Species of Concern     161
Endangered              16
Threatened              10
In Recovery              4
Name: conservation_status, dtype: int64

In [8]:
display(con_status_counts[1:].sum()/species.shape[0])
con_status_counts[0]/species.shape[0]

0.03279532967032967

0.9672046703296703

<font size="3">Only 3% of the data set contains various conservation statuses while 97% are NaN values. My initial guess is that these species are `not in danger` and are experiencing healthy numbers.<br><br>Let's do a little more investigation before making this conclusion.</font>

In [9]:
species.category.value_counts()

Vascular Plant       4470
Bird                  521
Nonvascular Plant     333
Mammal                214
Fish                  127
Amphibian              80
Reptile                79
Name: category, dtype: int64

<font size="3">Plants make up the vast majority of the species while the rest of the species are animals from various classes.</font>

In [10]:
display(species.scientific_name.nunique())
display(observations.scientific_name.nunique()) 
species.common_names.nunique()

5541

5541

5504

<font size="3">The dataframe observations only contains the scientific name and not the common name. There are 37 less total number of common names compared to scientific names, indicating that some species may share a common name. There are 5824 total rows in the data frame indicating 283 possible duplicates in the species scientific names.<br><br>The observations data frame also contains 5541 unique scientific names. To make sure all species are included in both, a check is performed to see if the unique names from both data sets are equal.</font>

In [11]:
# Casting the list to a set could also be used here
sorted(observations.scientific_name.unique()) == sorted(species.scientific_name.unique())

True

<font size="3">This indicates that all unqiue species are contained in both. Considering the fact that the vast majority of species have **NaN** for conservation status, and no species scientific names are missing in the observation data, it is safe to say that since all the animals and plants in the species data are being observed, that a missing conservation status means that the species is not under any threat.<br><br>Thus, the null values will be changed to a new category *"`not in danger`"* after some other analysis.<br><br> Null Values will be treated as a species not under conservation watch.</font>

In [12]:
# species.conservation_status.isna().sum()

In [13]:
# species.fillna('`not in danger`', inplace=True)
# species

In [14]:
species.conservation_status.value_counts().sum()

191

<font size="3">From looking at the value count, we know there are only 191 species have some kind of conservation status. For the sake of this analysis, I will consider this group with some negative conservation status as *`Under Watch`* or *`In Danger`*. This will be how I refer to these species.<br><br>
Let's look at the percentage of each species under watch based on their Class (or category in the data)</font>

In [15]:
cat_counts =species.category.value_counts().to_dict()

In [16]:
cat_counts =species.category.value_counts().to_dict()
under_watch = species.groupby(['category', 'conservation_status'])['scientific_name'].count().reset_index()
# could use a lambda function here
def percentages(row):
    return row['scientific_name']/cat_counts[row['category']]
under_watch['percentage'] = under_watch.apply(percentages, axis=1)
under_watch.sort_values(by='percentage', ascending=False)

,category,conservation_status,scientific_name,percentage
5,Bird,Species of Concern,72,0.138196
11,Mammal,Species of Concern,28,0.130841
14,Reptile,Species of Concern,5,0.063291
1,Amphibian,Species of Concern,4,0.050000
9,Mammal,Endangered,7,0.032710
7,Fish,Species of Concern,4,0.031496
8,Fish,Threatened,4,0.031496
2,Amphibian,Threatened,2,0.025000
6,Fish,Endangered,3,0.023622
13,Nonvascular Plant,Species of Concern,5,0.015015


<font size="3">It is clear from the chart that most species are `not in danger`. Of those that are, birds and mammals have about 13% as `species of concern`. Other statuses and classes of species are at 6% total or less. The other numbers are low, especially with plants where only around 1.5% of species are of concern or worse. 
<br><br>
Let's move on to investigate duplicate entries.</font>

In [17]:
species.duplicated().sum()

0

<font size="3">There are no complete duplicate rows indicating that any duplication is coming from the scientific name or common name only, as per our counts earlier. I suspect there are different common names for some species and some scientific names that are duplicated. 
<br><br>
Let's take a look at how many there are of each.</font>

In [18]:
species.duplicated(subset=['scientific_name']).sum(), species.duplicated(subset=['common_names']).sum()

(283, 320)

<font size="3">Let's see which share both scientific and common names</font>

In [19]:
species[species.duplicated(subset=['scientific_name','common_names'], keep=False)].sort_values(by='common_names')

,category,scientific_name,common_names,conservation_status
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered
560,Fish,Oncorhynchus mykiss,Rainbow Trout,NaN
3283,Fish,Oncorhynchus mykiss,Rainbow Trout,Threatened


<font size="3">Only two species, Canis lupus and Oncorhynchus mykiss, which I will refer in the future by the common names, have shared scientific and common names. In addition, they also have mixed statuses. I will come back and look in more detail at these two particular species<br><br>
Let's look next at species with duplicated common names first to see their conservation status.</font>

In [20]:
# pd.set_option('display.max_rows', 30)
duplicated_common_names = species[species.duplicated(subset=['common_names'], keep=False)]
duplicated_common_names.sort_values(by='common_names')#[~duplicated_common_names.conservation_status.isna()]

,category,scientific_name,common_names,conservation_status
2730,Nonvascular Plant,Dichodontium pellucidum,A Moss,NaN
2822,Nonvascular Plant,Cirriphyllum piliferum,A Moss,NaN
2022,Vascular Plant,Carex normalis,"A Sedge, Sedge",NaN
1971,Vascular Plant,Carex bromoides,"A Sedge, Sedge",NaN
1960,Vascular Plant,Carex annectens,"A Sedge, Sedge",NaN
...,...,...,...,...
250,Bird,Dendroica coronata,Yellow-Rumped Warbler,NaN
252,Bird,Dendroica dominica,Yellow-Throated Warbler,NaN
3206,Bird,Setophaga dominica,Yellow-Throated Warbler,NaN
2957,Nonvascular Plant,Zygodon viridissimus var. rupestris,Zygodon Moss,NaN


In [21]:
duplicated_common_names.conservation_status.value_counts(dropna=False)

NaN                   556
Species of Concern      9
In Recovery             1
Threatened              1
Endangered              1
Name: conservation_status, dtype: int64

<font size="3">The majority of the duplicated species are `not in danger`. Below, I will create a list of those species who are under watch and investigate if any have mixed statuses like the gray wolf or rainbow trout.</font>

In [22]:
display(duplicated_common_names[~duplicated_common_names.conservation_status.isna()].sort_values(by='common_names'))
duplicated_common_names[~duplicated_common_names.conservation_status.isna()].sort_values(by='common_names').shape[0]

,category,scientific_name,common_names,conservation_status
2929,Nonvascular Plant,Bazzania nudicaulis,Bazzania,Species of Concern
185,Bird,Guiraca caerulea,Blue Grosbeak,Species of Concern
2292,Vascular Plant,Poa paludigena,Bog Bluegrass,Species of Concern
3102,Bird,Stellula calliope,Calliope Hummingbird,Species of Concern
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered
3288,Fish,Cottus bairdii,Mottled Sculpin,Species of Concern
284,Bird,Vermivora ruficapilla,Nashville Warbler,Species of Concern
310,Bird,Contopus cooperi,Olive-Sided Flycatcher,Species of Concern
3283,Fish,Oncorhynchus mykiss,Rainbow Trout,Threatened


12

<font size="3">Only the two animals from before have statuses that are not `species of concern` and have duplicated common name. They also share the same species name. I can use this information to confirm that the other dupclicated common name species have mixed status with _``not in danger``_. <br><br>The gray wolf is included twice, so there should be 9 of them.</font>

In [23]:
mask = duplicated_common_names.conservation_status == 'Species of Concern'
spec_of_concern = duplicated_common_names[mask].common_names.to_list()
display(len(spec_of_concern))
duplicated_common_names[duplicated_common_names.common_names.isin(spec_of_concern)].sort_values(by='common_names')

9

,category,scientific_name,common_names,conservation_status
2930,Nonvascular Plant,Bazzania tricrenata,Bazzania,NaN
2929,Nonvascular Plant,Bazzania nudicaulis,Bazzania,Species of Concern
2928,Nonvascular Plant,Bazzania denudata,Bazzania,NaN
4525,Bird,Passerina caerulea,Blue Grosbeak,NaN
185,Bird,Guiraca caerulea,Blue Grosbeak,Species of Concern
2292,Vascular Plant,Poa paludigena,Bog Bluegrass,Species of Concern
5628,Vascular Plant,Poa leptocoma ssp. leptocoma,Bog Bluegrass,NaN
3102,Bird,Stellula calliope,Calliope Hummingbird,Species of Concern
4506,Bird,Selasphorus calliope,Calliope Hummingbird,NaN
563,Fish,Cottus bairdi,Mottled Sculpin,NaN


<font size="3">The numbers add up. My initial suspicion appears to be the case. Some of these animals or plants with different common names are variations on a species. The ones that are both ``not in danger`` and `species of concern` are technically different species. Thus, for all extent and purposes, they are separate entities with there own unqiue scientific name and conservation status. Furthermore, the majority are `not in danger`, while 11 of them have different converstation statuses. All will be kept as seperate entities since the scientific names are different and none of them will be dropped from the data.<br><br>
Next, let's look at the duplicated scientific names.</font>

In [24]:
duplicated_scientific_names = species[species.duplicated(subset=['scientific_name'], keep=False)].sort_values(by='scientific_name')
duplicated_scientific_names

,category,scientific_name,common_names,conservation_status
5553,Vascular Plant,Agrostis capillaris,"Colonial Bent, Colonial Bentgrass",NaN
2132,Vascular Plant,Agrostis capillaris,Rhode Island Bent,NaN
2134,Vascular Plant,Agrostis gigantea,Redtop,NaN
5554,Vascular Plant,Agrostis gigantea,"Black Bent, Redtop, Water Bentgrass",NaN
4178,Vascular Plant,Agrostis mertensii,"Arctic Bentgrass, Northern Bentgrass",NaN
...,...,...,...,...
5643,Vascular Plant,Vulpia myuros,"Foxtail Fescue, Rattail Fescue, Rat-Tail Fescu...",NaN
2331,Vascular Plant,Vulpia octoflora,Annual Fescue,NaN
4290,Vascular Plant,Vulpia octoflora,"Eight-Flower Six-Weeks Grass, Pullout Grass, S...",NaN
3347,Vascular Plant,Zizia aptera,"Heartleaf Alexanders, Heart-Leaf Alexanders, M...",NaN


In [25]:
display(duplicated_scientific_names[['conservation_status']].value_counts(dropna=False))
duplicated_scientific_names[['conservation_status']].value_counts().sum()

conservation_status
NaN                    534
Species of Concern      19
Endangered               2
In Recovery              1
Threatened               1
dtype: int64

23

<font size="3">Some species with duplicated scientific names are in the database twice because they have different common names with some having more than one common name with another. Like with common names, a majority of them are `not in danger`. It is possible that these should be considered as different entities since data collection might be occurring on different groups of these species or in different locations, but we are not sure yet. <br><br>
Let's focus on that number 23. To start, we look at the conservation status of these species and counts to see if there are any that have more than 3 entries.</font>

In [26]:
display(duplicated_scientific_names.groupby(['conservation_status', 'scientific_name'])['common_names'].count().sum())
display(duplicated_scientific_names.groupby(['conservation_status', 'scientific_name'])['common_names'].count())
print("\nValue Counts")
duplicated_scientific_names.scientific_name.value_counts().value_counts()

23

conservation_status  scientific_name          
Endangered           Canis lupus                  2
In Recovery          Canis lupus                  1
Species of Concern   Eptesicus fuscus             2
                     Gavia immer                  2
                     Lasionycteris noctivagans    2
                     Myotis californicus          2
                     Myotis lucifugus             3
                     Nycticorax nycticorax        2
                     Pandion haliaetus            2
                     Riparia riparia              2
                     Taxidea taxus                2
Threatened           Oncorhynchus mykiss          1
Name: common_names, dtype: int64


Value Counts


2    265
3      9
Name: scientific_name, dtype: int64

In [27]:
duplicated_scientific_names.groupby(['conservation_status', 'scientific_name'])['common_names'].count().index.get_level_values('scientific_name').nunique()

11

<font size="3">All duplicated species by scientific name only have 2 or 3 duplicate entries. The number of species under watch is 11. Only 9 species total have 3 duplicated scientific names whereas 265 have 2 duplicated names. It appears that the Canis lupus (the gray wolf) and Myotis lucifugus (the little brown bat) both have three statuses under watch, with the little brown bat having all three as as `species of concern`. We already know the gray wolf and rainbow trout have mixed statuses. That leaves us with the remaining 9 of 11. So what about the duplicated species with 2 entries? Are any of them 1 of the 7 to remaning on the 3 species list? <br><br>
Let's look more at the species on this list to see which with a count of 2 have a mixed statuses and confirm the statuses of species with 3 entries.</font>

In [28]:
list_of_3_entries = duplicated_scientific_names.scientific_name.value_counts()[duplicated_scientific_names.scientific_name.value_counts() == 3].index
list_of_2_entries = duplicated_scientific_names.scientific_name.value_counts()[duplicated_scientific_names.scientific_name.value_counts() == 2].index

In [29]:
duplicated_scientific_names[duplicated_scientific_names.scientific_name.isin(list_of_3_entries)]

,category,scientific_name,common_names,conservation_status
8,Mammal,Canis lupus,Gray Wolf,Endangered
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered
49,Mammal,Castor canadensis,"American Beaver, Beaver",NaN
4475,Mammal,Castor canadensis,Beaver,NaN
3050,Mammal,Castor canadensis,American Beaver,NaN
4513,Bird,Columba livia,Rock Pigeon,NaN
3138,Bird,Columba livia,"Common Pigeon, Rock Dove, Rock Pigeon",NaN
156,Bird,Columba livia,Rock Dove,NaN
4236,Vascular Plant,Holcus lanatus,"Common Velvetgrass, Yorkshire-Fog",NaN


<font size="3"> It is confirmed that none of the list of 3 except the wolf and bat have mixed statuses. Therefore, the remaining 9 species (11 including ranibow trout, gray wolf and brown bat) must have only 2 entries each.<br><br>Since the list of 2 is large (265), let's create a mask to filter out any species that have both statuses as _`not in danger`_. We will test the mask first with the list of 3.</font>

In [30]:
# duplicated_scientific_names[~(duplicated_scientific_names.conservation_status == "`not in danger`") & duplicated_scientific_names.scientific_name.isin(list_of_3_entries)]
mask_3 = ~(duplicated_scientific_names.conservation_status.isna()) & duplicated_scientific_names.scientific_name.isin(list_of_3_entries)
duplicated_scientific_names[mask_3]#.scientific_name.nunique()

,category,scientific_name,common_names,conservation_status
8,Mammal,Canis lupus,Gray Wolf,Endangered
3020,Mammal,Canis lupus,"Gray Wolf, Wolf",In Recovery
4448,Mammal,Canis lupus,"Gray Wolf, Wolf",Endangered
4467,Mammal,Myotis lucifugus,Little Brown Myotis,Species of Concern
3042,Mammal,Myotis lucifugus,"Little Brown Bat, Little Brown Myotis, Little ...",Species of Concern
37,Mammal,Myotis lucifugus,"Little Brown Bat, Little Brown Myotis",Species of Concern


In [31]:
# duplicated_scientific_names[~(duplicated_scientific_names.conservation_status == "`not in danger`") & duplicated_scientific_names.scientific_name.isin(list_of_2_entries)]
mask_2 = ~(duplicated_scientific_names.conservation_status.isna()) & duplicated_scientific_names.scientific_name.isin(list_of_2_entries)
duplicated_scientific_names[mask_2]

,category,scientific_name,common_names,conservation_status
29,Mammal,Eptesicus fuscus,Big Brown Bat,Species of Concern
3035,Mammal,Eptesicus fuscus,"Big Brown Bat, Big Brown Bat",Species of Concern
3150,Bird,Gavia immer,"Common Loon, Great Northern Diver, Great North...",Species of Concern
172,Bird,Gavia immer,Common Loon,Species of Concern
30,Mammal,Lasionycteris noctivagans,Silver-Haired Bat,Species of Concern
3037,Mammal,Lasionycteris noctivagans,"Silver-Haired Bat, Silver-Haired Bat",Species of Concern
4465,Mammal,Myotis californicus,California Myotis,Species of Concern
3039,Mammal,Myotis californicus,"California Myotis, California Myotis, Californ...",Species of Concern
337,Bird,Nycticorax nycticorax,Black-Crowned Night-Heron,Species of Concern
4564,Bird,Nycticorax nycticorax,Black-Crowned Night Heron,Species of Concern


<font size="3">Because there are two entries for each `species of concern` in this filtered data, none of the entries under watch have a mixed status, except for the rainbow trout. That means the majority of entries of 2 have a status of `not in danger`.<br><br> Finally, I will use my masks to make sure all 11 species with duplicates under watch were accounted for.</font>

In [32]:
duplicated_scientific_names[mask_2 + mask_3].scientific_name.nunique()

# display(sorted(duplicated_scientific_names[mask].scientific_name.unique()))
# sorted(duplicated_scientific_names.groupby(['conservation_status', 'scientific_name'])['common_names'].count().to_frame().index.get_level_values(1).unique()[1:])

11

<font size="3">In summary: <ul><li>All duplicated species are either on the list 2 or 3 times.</li><li>Almost all of the species with multiple entries have the same conservation status. The majority here are _`not in danger`_</li><li>**11** species are _in danger_ or _under watch_.</li><li> **9 of 11** have a status of 'species of concern. Of those 9, *only the little brown bat has 3 duplicates* while the rest have 2.</li><li>Only **2 of 11** species, *the gray wolf and rainbow trout*, have multiple or mixed statuses.</li></ul><br>
I conclude that the duplicate entries are most likely not by mistake. They might be included more than once because of a previous change in status. The authorities are constantly updating the conservation status of species. Alternatively, it is possible that there were different pools or populations of the species being monitored in different locations, or they were being monitored by different observers. This might explain the different common names listed. Since there is no date data, it is not possible to make a definitive conclusion.
<br><br>
It is crucial to look at the obervations table to get a sense of what is going with the multiple entries.</font>

<font size="3">Before that, I am going to save the data frame containing all the duplicate species with mixed status</font>

In [33]:
duplicated_scientific_names_under_watch = duplicated_scientific_names[~duplicated_scientific_names.conservation_status.isna()]['scientific_name'].unique().tolist()
danger = duplicated_scientific_names[duplicated_scientific_names.scientific_name.apply(lambda species: species in duplicated_scientific_names_under_watch)].reset_index()
danger.to_csv('data/species_in_danger.csv')

In [34]:
# FOR DROPPING 

# duplicated_mask = species.duplicated(subset=['scientific_name'], keep=False)
# trout_keep_NA_mask = species.scientific_name == species.loc[560, 'scientific_name']
# duplicated_mask_NA = species.conservation_status.isna() & ~trout_keep_NA_mask

# TEST = species[~(duplicated_mask & duplicated_mask_NA)]
# species[species.conservation_status.isna()].drop_duplicates(subset=['scientific_name'], keep='first')
# duplicated_scientific_names_mixed_status = duplicated_scientific_names[~duplicated_scientific_names.conservation_status.isna()]['scientific_name'].unique().tolist()
# species[species.scientific_name.isin(duplicated_scientific_names_mixed_status)]

In [35]:
display(observations.shape)
observations.head()

(23296, 3)

,scientific_name,park_name,observations
0,Vicia benghalensis,Great Smoky Mountains National Park,68
1,Neovison vison,Great Smoky Mountains National Park,77
2,Prunus subcordata,Yosemite National Park,138
3,Abutilon theophrasti,Bryce National Park,84
4,Githopsis specularioides,Great Smoky Mountains National Park,85


<font size="3">The observations data contains only the scientific name, park name, and the number of observations. There is no date or time data included, nor data about the common name. Also, it is unclear if an observation is a unique count of a species or just a sighting of a species, but I assume it is the case that an observation is a unqiue number of the species found in the park at a given time.</font>

In [36]:
observations.duplicated().sum()

15

In [37]:
observations[observations.duplicated(keep=False)].sort_values(by='scientific_name')#.head()

,scientific_name,park_name,observations
513,Arctium minus,Yosemite National Park,162
10674,Arctium minus,Yosemite National Park,162
4527,Botrychium virginianum,Yellowstone National Park,232
20294,Botrychium virginianum,Yellowstone National Park,232
19392,Cichorium intybus,Yellowstone National Park,266
14142,Cichorium intybus,Yellowstone National Park,266
7263,Echinochloa crus-galli,Great Smoky Mountains National Park,62
1454,Echinochloa crus-galli,Great Smoky Mountains National Park,62
1020,Eleocharis palustris,Great Smoky Mountains National Park,62
12381,Eleocharis palustris,Great Smoky Mountains National Park,62


<font size="3">Given the large number of observations, it is safe to say that duplicates are just measures of a species with the same number taken at different times or by different observers. It would mean that the population numbers is stable for that species in that park over some time period, or that it fluctuated and returned to a previous population number<br><br>Next, I want to look at the total numbers and counts of some aggregated data to confirm a suspicion: **each species is measured once at each park and four times in total**.</font>

In [38]:
species.shape[0] * 4 == observations.shape[0]

True

<font size="3">The number in the obserations data set is 4 times the number in the species data set. This makes sense that each speicies from the data set is observed in the four different parks, assumingly at the same time frame.<br><br>This will be confirmed with a check below.</font>f

In [39]:
sorted_species_park = observations.sort_values(by=['scientific_name', 'park_name'])
display(sorted_species_park.tail(12))
sorted_species_park.head(12)

,scientific_name,park_name,observations
5350,Zonotrichia querula,Bryce National Park,105
15795,Zonotrichia querula,Great Smoky Mountains National Park,83
17507,Zonotrichia querula,Yellowstone National Park,268
12923,Zonotrichia querula,Yosemite National Park,160
16130,Zygodon viridissimus,Bryce National Park,100
3649,Zygodon viridissimus,Great Smoky Mountains National Park,71
5015,Zygodon viridissimus,Yellowstone National Park,270
5539,Zygodon viridissimus,Yosemite National Park,159
12775,Zygodon viridissimus var. rupestris,Bryce National Park,102
20040,Zygodon viridissimus var. rupestris,Great Smoky Mountains National Park,102


,scientific_name,park_name,observations
4968,Abies bifolia,Bryce National Park,109
17990,Abies bifolia,Great Smoky Mountains National Park,72
8050,Abies bifolia,Yellowstone National Park,215
20930,Abies bifolia,Yosemite National Park,136
8170,Abies concolor,Bryce National Park,83
4647,Abies concolor,Great Smoky Mountains National Park,101
15193,Abies concolor,Yellowstone National Park,241
23166,Abies concolor,Yosemite National Park,205
2014,Abies fraseri,Bryce National Park,109
5112,Abies fraseri,Great Smoky Mountains National Park,81


fs!This also implies that the duplicates in the species with scientific names data are not by mistake, but are seperate sets of observation measures.<br><br>Let's do some confirmation of this by looking at the value counts of value counts. *We know from are dupcliate analyis that the duplicates were in groups of 3 or 2*, so the observations should only have values of 4, 8 and 12.</font>

In [40]:
obsv_counts = observations.scientific_name.value_counts()
display(obsv_counts)
display(obsv_counts.to_frame().value_counts())
display(sorted_species_park[['scientific_name', 'park_name']].value_counts().value_counts())
all(obsv_counts[obsv_counts.values > 4].index.sort_values() == duplicated_scientific_names.scientific_name.unique())

Myotis lucifugus                        12
Puma concolor                           12
Hypochaeris radicata                    12
Holcus lanatus                          12
Streptopelia decaocto                   12
                                        ..
Packera dimorphophylla var. paysonii     4
Smilax bona-nox                          4
Chondestes grammacus                     4
Leymus triticoides                       4
Dichanthelium depauperatum               4
Name: scientific_name, Length: 5541, dtype: int64

scientific_name
4                  5267
8                   265
12                    9
dtype: int64

1    21068
2     1060
3       36
dtype: int64

True

<font size="3">We confirmed the names are the same and the counts match (9 species had duplicates of 3 and 265 had duplicates of 2). Sorting by name and parks also confirms that there are four observations for all species.<br><br>Therefore, we should keep all the data in the two data sets. This will cause some problems for merging the data set, though.</font>

<font size="3">Let's look at the observation data to get an idea of the meaning of the different conservation statuses. We can filter data for the duplicates using the observation counts data frame.</font>

In [41]:
for
# duplicated_scientific_names[duplicated_scientific_names.scientific_name.isin(obsv_counts[:9].index)].sort_values(by='scientific_name')

SyntaxError: invalid syntax (<ipython-input-41-cb8f7c949f5b>, line 1)

In [60]:
in_danger = pd.read_csv('data/species_in_danger.csv', index_col=0)

In [49]:
display(observations[observations.scientific_name.isin(obsv_counts[:9].index)].reset_index().sort_values(by=['scientific_name', 'index']).head(12))
observations[observations.scientific_name.isin(obsv_counts[:9].index)].reset_index().sort_values(by=['scientific_name', 'index']).tail(24)

,index,scientific_name,park_name,observations
4,1294,Canis lupus,Yosemite National Park,35
9,1766,Canis lupus,Bryce National Park,27
35,7346,Canis lupus,Bryce National Park,29
53,9884,Canis lupus,Bryce National Park,74
55,10190,Canis lupus,Great Smoky Mountains National Park,15
56,10268,Canis lupus,Yellowstone National Park,60
63,10907,Canis lupus,Yellowstone National Park,67
72,13427,Canis lupus,Yellowstone National Park,203
92,17756,Canis lupus,Great Smoky Mountains National Park,14
98,19330,Canis lupus,Yosemite National Park,117


,index,scientific_name,park_name,observations
0,77,Puma concolor,Bryce National Park,92
14,2649,Puma concolor,Great Smoky Mountains National Park,79
18,3270,Puma concolor,Yellowstone National Park,243
31,6660,Puma concolor,Great Smoky Mountains National Park,48
46,8976,Puma concolor,Yosemite National Park,121
47,9298,Puma concolor,Yosemite National Park,165
49,9605,Puma concolor,Yellowstone National Park,240
58,10388,Puma concolor,Bryce National Park,128
81,15676,Puma concolor,Yellowstone National Park,270
88,16551,Puma concolor,Great Smoky Mountains National Park,112


fs!A quick look shows that there is not particular order to the data that would help us seperate the multiple labelled species. That is, it is impossible to tell which set of observations match with the species with unique common names and the same scientific name. I kept the original indexing to see if that would maintain some sort of order in the parks, which could represent observations being made on certain days in a certain order. I was hoping to see if I could reorder or make conclusions about these multilisted species.br!This is not such a problem since most of this species maintain the same status. The exception is when we are dealing with species that had multiple statuses, the rainbow trout and gray wolf. The rest of the duplicates have the same labelled conservation statusfe!

In [ ]:
species[species.scientific_name.isin(list_of_3_entries)].sort_values(by='scientific_name')

In [ ]:
print(f'mode\t{observations.observations.mode()}')
print(observations.observations.agg(['max', 'min', 'median', 'mean']))

In [43]:
duplicated_scientific_names_under_watch

['Canis lupus',
 'Eptesicus fuscus',
 'Gavia immer',
 'Lasionycteris noctivagans',
 'Myotis californicus',
 'Myotis lucifugus',
 'Nycticorax nycticorax',
 'Oncorhynchus mykiss',
 'Pandion haliaetus',
 'Riparia riparia',
 'Taxidea taxus']

In [57]:
# pd.set_option('display.max_rows', 100)

under_watch =observations[observations.scientific_name.apply(lambda species: species in duplicated_scientific_names_under_watch)].sort_values(by=['scientific_name', 'park_name'])

In [64]:
in_danger.tail()

,index,category,scientific_name,common_names,conservation_status
19,104,Bird,Pandion haliaetus,Osprey,Species of Concern
20,226,Bird,Riparia riparia,Bank Swallow,Species of Concern
21,3185,Bird,Riparia riparia,"Bank Swallow, Sand Martin",Species of Concern
22,3029,Mammal,Taxidea taxus,"American Badger, Badger",Species of Concern
23,4457,Mammal,Taxidea taxus,Badger,Species of Concern


In [63]:
under_watch.tail(16)

,scientific_name,park_name,observations
872,Riparia riparia,Bryce National Park,88
1259,Riparia riparia,Bryce National Park,91
2672,Riparia riparia,Great Smoky Mountains National Park,92
4916,Riparia riparia,Great Smoky Mountains National Park,76
220,Riparia riparia,Yellowstone National Park,199
14216,Riparia riparia,Yellowstone National Park,226
7545,Riparia riparia,Yosemite National Park,125
18275,Riparia riparia,Yosemite National Park,145
1581,Taxidea taxus,Bryce National Park,94
11772,Taxidea taxus,Bryce National Park,65


,index,category,scientific_name,common_names,conservation_status
19,104,Bird,Pandion haliaetus,Osprey,Species of Concern
20,226,Bird,Riparia riparia,Bank Swallow,Species of Concern
21,3185,Bird,Riparia riparia,"Bank Swallow, Sand Martin",Species of Concern
22,3029,Mammal,Taxidea taxus,"American Badger, Badger",Species of Concern
23,4457,Mammal,Taxidea taxus,Badger,Species of Concern


fs!Once a species is over 250 in observation, they are no longer in danger or or concern. Threatened appears to be under 100 observations in 3 or more parks while Under Concern seems to be under 100 in 2 or more parksfe!

In [65]:
species_2 = species[~(species.scientific_name.isin(['Canis lupus', 'Oncorhynchus mykiss']))]

In [95]:
display(in_danger.scientific_name.unique())
vc = species_2.groupby(['scientific_name', 'conservation_status']).count()
vc[vc.category > 1]

array(['Canis lupus', 'Eptesicus fuscus', 'Gavia immer',
       'Lasionycteris noctivagans', 'Myotis californicus',
       'Myotis lucifugus', 'Nycticorax nycticorax', 'Oncorhynchus mykiss',
       'Pandion haliaetus', 'Riparia riparia', 'Taxidea taxus'],
      dtype=object)

,,category,common_names
scientific_name,conservation_status,,
Eptesicus fuscus,Species of Concern,2,2
Gavia immer,Species of Concern,2,2
Lasionycteris noctivagans,Species of Concern,2,2
Myotis californicus,Species of Concern,2,2
Myotis lucifugus,Species of Concern,3,3
Nycticorax nycticorax,Species of Concern,2,2
Pandion haliaetus,Species of Concern,2,2
Riparia riparia,Species of Concern,2,2
Taxidea taxus,Species of Concern,2,2


fs!These match up. It is safe to drop the duplicates since all other species match to only a single convservation statusfe!

In [103]:
species_2.drop_duplicates(subset='scientific_name', inplace=True)

/Users/robertjfpal/opt/anaconda3/envs/the_way/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [105]:
observations_2 = observations[~(observations.scientific_name.isin(['Canis lupus', 'Oncorhynchus mykiss']))]

In [117]:
display(species_2.shape[0] * 4)
observations_2.shape[0] - 264*4 - 8*8

22156

22156

In [114]:
biodiversity = species_2.merge(observations_2, on='scientific_name', how='left')

In [125]:
# biodiversity[biodiversity.scientific_name.isin(duplicated_scientific_names_under_watch)].head(36)

In [127]:
biodiversity.fillna('Not in Danger', inplace=True)

In [131]:
biodiversity.to_csv('biodiversity.csv')

In [ ]:
##### CONCATTING

In [ ]:
species_df = pd.concat([species, species, species, species], ignore_index=True)

In [ ]:
species_df = species_df.sort_values(by=['scientific_name', 'common_names']).reset_index(drop=True)#.drop(columns='scientific_name')
# species_df.drop(columns='scientific_name', inplace=True)

In [ ]:
species_df[species_df.scientific_name == 'Canis lupus']

In [ ]:
observations_df = observations.sort_values(by=['scientific_name']).reset_index(drop=True)

In [ ]:
observations[observations.scientific_name == 'Canis lupus'].sort_values(by=['park_name', 'observations'])

In [ ]:
species_df = species_df.drop(columns='scientific_name')
df = pd.concat([species_df, observations_df], axis=1)
df

In [ ]:
df[df.scientific_name == 'Canis lupus'].sort_values(by=['common_names', 'conservation_status'])

In [ ]:
####### STOP HERE

It appears that the Gray Wolf was endangered but recovered, while the Rainbow Trout were no longer under threat or of concern, at least in the 4 National Parks under study

In [ ]:
observations[observations.scientific_name == 'Canis lupus'].sort_values(by='observations')

In [ ]:
observations[observations.scientific_name == 'Oncorhynchus mykiss'].sort_values(by='observations')

In [ ]:
trout_ob = observations[observations.scientific_name == 'Oncorhynchus mykiss'].sort_values(by='observations')
trout_sp = species[species.scientific_name == 'Oncorhynchus mykiss']

In [ ]:
trout_sp

In [ ]:
trout_ob

In [ ]:
trout = pd.merge(trout_sp, trout_ob, how='cross')

In [ ]:
trout[(trout.conservation_status.isna()) & (trout.observations > 200) | ~trout.conservation_status.isna()]


In [ ]:
df = duplicated_scientific_names.merge(observations, on='scientific_name')

In [ ]:
# df.drop_duplicates(subset=['scientific_name', 'observations']).head(20)

In [ ]:
# observations[observations.duplicated(subset=['scientific_name'], keep=False)].sort_values(by='scientific_name')

The names are equal in bddoth data sets, so there would be no species that are in one but not in another

Looking at duplicated rows
Figure out if the duplicates are errors or not

In [ ]:
multiple_species = species.scientific_name.value_counts()

In [ ]:
multiple_species.where(multiple_species > 1, inplace=True)

In [ ]:
multiple_species.dropna(inplace=True)

In [ ]:
multiple_species.value_counts()

In [ ]:
species_changed_status = multiple_species.index.tolist()

In [ ]:
species_df = species[~species.scientific_name.isin(species_changed_status)]

In [ ]:
species_df.shape

In [ ]:
species_df.duplicated().sum()

In [ ]:
(species_df.groupby('scientific_name').count()['conservation_status'] == 1).sum()

In [ ]:
# species_df[~(species_df.conservation_status == "`not in danger`")]
species_df.head()

In [ ]:
observations.head()

In [ ]:
observations.shape

Data merging safely from here

In [ ]:
right = species_df.merge(observations, on='scientific_name', how='right').sort_values(by=['scientific_name', 'observations']).reset_index(drop=True)

In [ ]:
left = species_df.join(observations.set_index('scientific_name'), on='scientific_name', how='left').sort_values(by=['scientific_name', 'observations']).reset_index(drop=True)

In [ ]:
right[right.scientific_name.isin(species_changed_status)]

In [ ]:
len(species_changed_status)